In [15]:
pip install --upgrade bebi103

Requirement already up-to-date: bebi103 in c:\users\kevin\anaconda3\lib\site-packages (0.0.52)
Note: you may need to restart the kernel to use updated packages.


In [16]:
import glob
import os

import numpy as np
import pandas as pd
import scipy.signal

# Image processing tools
import skimage
import skimage.io

import bebi103

import bokeh
bokeh.io.output_notebook()

Loading BokehJS ...

In [17]:
# The directory containing daytime data
data_dir = 'C:/Users/kevin/BME202/t04/cassiopea_pulsation/day'

# Glob string for images
im_glob = os.path.join(data_dir, '*.TIF')

# Get list of files in directory
im_list = sorted(glob.glob(im_glob))

# Let's look at the first 10 entries
im_list[:10]

['C:/Users/kevin/BME202/t04/cassiopea_pulsation/day\\Frame_545000.TIF',
 'C:/Users/kevin/BME202/t04/cassiopea_pulsation/day\\Frame_545001.TIF',
 'C:/Users/kevin/BME202/t04/cassiopea_pulsation/day\\Frame_545002.TIF',
 'C:/Users/kevin/BME202/t04/cassiopea_pulsation/day\\Frame_545003.TIF',
 'C:/Users/kevin/BME202/t04/cassiopea_pulsation/day\\Frame_545004.TIF',
 'C:/Users/kevin/BME202/t04/cassiopea_pulsation/day\\Frame_545005.TIF',
 'C:/Users/kevin/BME202/t04/cassiopea_pulsation/day\\Frame_545006.TIF',
 'C:/Users/kevin/BME202/t04/cassiopea_pulsation/day\\Frame_545007.TIF',
 'C:/Users/kevin/BME202/t04/cassiopea_pulsation/day\\Frame_545008.TIF',
 'C:/Users/kevin/BME202/t04/cassiopea_pulsation/day\\Frame_545009.TIF']

In [18]:
# Read in the image using skimage
im = skimage.io.imread(im_list[0])

# Let's get information about the image
print(type(im), im.dtype, im.shape)

<class 'numpy.ndarray'> uint16 (480, 640, 3)


In [19]:
# Test to see if each R, G, and B value is the same
((im[:,:,0] == im[:,:,1]) &  (im[:,:,1] == im[:,:,2])).all()

True

In [20]:
# Just slice the red channel
im = im[:,:,0]

In [21]:
p = bebi103.image.imshow(im)
bokeh.io.show(p)

In [28]:
click_positions = bebi103.image.record_clicks(im, flip=False)

In [13]:
df_clicks = click_positions.to_df()

# Take a look
df_clicks

,x,y


In [12]:
box_length = np.sqrt(
    (df_clicks['x'][1] - df_clicks['x'][0])**2 
    + (df_clicks['y'][1] - df_clicks['y'][0])**2
)

IndexError: index out of bounds

In [14]:
interpixel_distance = 4 / box_length * 2.54

NameError: name 'box_length' is not defined

In [ ]:
p = bebi103.image.imshow(im,
                       interpixel_distance=interpixel_distance,
                       length_units='cm')
bokeh.io.show(p)

In [ ]:
ic = skimage.io.ImageCollection(im_glob, conserve_memory=True)

In [ ]:
len(ic)

In [ ]:
def squish_rgb(fname, **kwargs):
    """
    Only take one channel. (Need to explicitly have the **kwargs to play
    nicely with skimage.io.ImageCollection.)
    """
    im = skimage.io.imread(fname)
    return im[:,:,0]

ic = skimage.io.ImageCollection(im_glob,
                                conserve_memory=True, 
                                load_func=squish_rgb)

In [ ]:
fps = 15
t = np.arange(0, len(ic)) / fps

In [ ]:
rois = bebi103.image.draw_rois(im, flip=False)

In [ ]:
df_rois = bebi103.image.roicds_to_df(rois)

# Take a look
df_rois

In [ ]:
df_rois.to_csv('t03c_sample_rois.csv', index=False)

In [ ]:
bebi103.image.verts_to_roi?

In [ ]:
df_rois.head()

In [ ]:
im.shape

In [ ]:
rois = [bebi103.image.verts_to_roi(g[['x', 'y']].values, *im.shape) 
            for _, g in df_rois.groupby('roi')]

In [ ]:
# Use the rightmost ROI
roi, roi_bbox, roi_box = rois[1]

# Make grayscale image that is now RGB
im = np.dstack(3*[skimage.img_as_float(ic[0])])

# Max out blue channel
im[roi, 2] = skimage.dtype_limits(im)[1]

# Look at the image
p = bebi103.image.imshow(im, cmap='rgb')
bokeh.io.show(p)

In [ ]:
# Get cropped image and ROI within it
im = ic[0][roi_bbox]
im_cropped_roi = roi_box

# Make grayscale image that is now RGB
im = np.dstack(3*[skimage.img_as_float(im)])

# Max out blue channel
im[im_cropped_roi,2] = skimage.dtype_limits(im)[1]

# Look at the image
p = bebi103.image.imshow(im, cmap='rgb')
bokeh.io.show(p)

In [ ]:
# Define load_func that only gives region bounding ROI
def load_roi(fname, roi_bbox=None, **kwargs):
    """
    Image loading function to only load ROI.
    """
    if roi_bbox is None:
        return skimage.io.imread(fname)[:, :, 0]
    else:
        return skimage.io.imread(fname)[:, :, 0][roi_bbox]


# Load image collection
ic = skimage.io.ImageCollection(
    im_glob, conserve_memory=True, load_func=load_roi, roi_bbox=roi_bbox
)

# Look at first image
p = bebi103.image.imshow(ic[0])
bokeh.io.show(p)

In [ ]:
# Set up NumPy array to store total pixel intensity
total_int = np.empty(len(t))

# Look through and compute total intensity in the ROI
for i, im in enumerate(ic):
    total_int[i] = ic[i][roi_box].sum()

In [ ]:
p = bokeh.plotting.figure(plot_width=650,
                          plot_height=300,
                          x_axis_label='time (s)', 
                          y_axis_label='total intensity')

p.line(t, total_int, line_width=2, line_join='bevel')

bokeh.io.show(p)

In [ ]:
total_int -= total_int.mean()
total_int = 1 + 2 * (
    (total_int - total_int.max()) 
                    / (total_int.max() - total_int.min())
)

p = bokeh.plotting.figure(
    plot_width=650,
    plot_height=300,
    x_axis_label='time (s)', 
    y_axis_label='total intensity'
)

p.line(t, total_int, line_width=2, line_join='bevel')

bokeh.io.show(p)

In [ ]:
#Creating a table of values for individual jellyfish
#Instructions to add information:
#replace "nightinterval" with an array that contains the 8 average peak interval times for each jellyfish, OR replace "nightinterval" in the indidata list.
#Do this for day interval, night inten (intensities), and day inten.
nightinterval = [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0]
dayinterval = [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0]
nightinten = [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0]
dayinten = [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0]



jellynames = ['jelly1', 'jelly2', 'jelly3', 'jelly4', 'jelly5', 'jelly6', 'jelly7', 'jelly8']

jelly = {
    'Jellyfish': ['jelly1', 'jelly2', 'jelly3', 'jelly4', 'jelly5', 'jelly6', 'jelly7', 'jelly8'],
}

indidata = list(zip(jellynames, nightinterval, dayinterval, nightinten, dayinten))

inditable = pd.DataFrame(indidata, columns = ['Jellyfish','Night Interval', 'Day Interval', 'Night Intensities', 'Day Intensities'])
inditable

In [ ]:
#creating overall table of means of each day/night group
#Instructions:
#Set equal the values of total nighttime mean peak interval values with nightintervaltot, and the nightinterstdev with the nighttime mean peak interval st. dev.
#repeat for the remaining values.

#total peak interval times across day/night
nightintervaltot = 1.0
dayintervaltot = 2.0

#standard deviation of the peak interval means
nightinterstdev = 1
dayinterstdev = 2

#Total intensity times across day/night
nightintentot = 3.0
dayintentot = 4.0

#st. dev of the intensities across day/night
nightintenstdev = 5.0
dayintenstdev = 6.0

meannames = ['Night Fish','Day Fish']
meaninter = [nightintervaltot, dayintervaltot]
interstdev = [nightinterstdev, dayinterstdev]
meaninten = [nightintentot, dayintentot]
intenstdev = [nightintenstdev, dayintenstdev]

meandata = list(zip(meannames, meaninter, interstdev, meaninten, intenstdev))

meantable = pd.DataFrame(meandata, columns = ['Jellyfish','Average Interval', 'St. Deviation', 'Average Intensities', 'St. Deviation'])
meantable

In [ ]:
>>> tbdayinter = [dayintervaltot, dayinterstdev]
>>> tbnightinter = [nightintervaltot, nightinterstdev]
>>> index = ['Mean Interval', 'Interval St. Dev']
>>> df = pd.DataFrame({'Nighttime': tbnightinter,
...                    'Daytime': tbdayinter}, index=index)
>>> ax = df.plot.bar(rot=0)


In [ ]:
>>> tbdayinten = [dayintentot, dayintenstdev]
>>> tbnightinten = [nightintentot, nightintenstdev]
>>> index = ['Mean Intensity', 'Intensity St. Dev']
>>> df = pd.DataFrame({'Nighttime': tbnightinten,
...                    'Daytime': tbdayinten}, index=index)
>>> ax = df.plot.bar(rot=0)